In [408]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

## 1. Determine the five most common journals and the total articles for each

## 2. Calculate the mean, median, and standard deviation of the open-access cost per article for each journal

In [409]:
# load dataset to clean:
df = pd.read_csv('/Users/evarubin/Dev/thinkful/data/bootcamp/unit1-lesson3-data-cleaning-validation-WELLCOME.csv', sep='\t')

In [410]:
# see columns
df.columns

Index(['PMID/PMCID', 'Publisher', 'Journal title', 'Article title',
       'COST (ﾣ) charged to Wellcome (inc VAT when charged)'],
      dtype='object')

In [411]:
# rename columns for easier manipulation
df = df.rename(index=str, columns={"PMID/PMCID": "Publisher_id", 
                                   "COST (ﾣ) charged to Wellcome (inc VAT when charged)": "Cost"})

### 1. Top journal titles and number of articles

In [412]:
# Get the top five most common journals before cleaning (showed PLOS and Nucleic Acids Research were not originally combined )
df['Journal title'].value_counts().head()

PLoS One                                           92
PLoS ONE                                           62
Journal of Biological Chemistry                    48
Nucleic Acids Research                             21
Proceedings of the National Academy of Sciences    19
Name: Journal title, dtype: int64

In [413]:
# Combine PLOS One to get accurate value_count()
df.loc[df['Journal title'].str.contains('PLOS ONE', case=False, na=False), 'Journal title'] = 'PLOS One'
df.loc[df['Journal title'].str.contains('PLOSONE', case=False, na=False), 'Journal title'] = 'PLOS One'
df.loc[df['Journal title'].str.contains('PLOS  ONE', case=False, na=False), 'Journal title'] = 'PLOS One'
df.loc[df['Journal title'].str.contains('PLOS 1', case=False, na=False), 'Journal title'] = 'PLOS One'
df.loc[df['Journal title'].str.contains('Public Library of Science ONE', case=False, na=False), 'Journal title'] = 'PLOS One'
df.loc[df['Journal title'].str.contains('PONE-D12-17947', case=False, na=False), 'Journal title'] = 'PLOS One'

In [414]:
# Combine Nucleic Acids Research to get accurate value_count().  Include all plos without "One" since plos has no journal named "plos"
df.loc[df['Journal title'].str.contains('nucleic acids research', case=False, na=False), 'Journal title'] = 'Nucleic Acids Research'
df.loc[df['Journal title'].str.contains('nucleic acid research', case=False, na=False), 'Journal title'] = 'Nucleic Acids Research'

In [415]:
# Biochemistry and Developmnt Counts are across publishers with journals that have the same name but are distinct journals
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(df.groupby(['Publisher','Journal title']).agg({'Article title': 'count'}))

                                                                                                       Article title
Publisher                                          Journal title                                                    
ACS                                                Biomacromolecules                                               1
                                                   J Med Chem                                                      2
                                                   J Org Chem                                                      1
                                                   Journal of Medicinal Chemistry                                  1
                                                   Journal of Proteome Research                                    1
                                                   Mol Pharm                                                       1
ACS (Amercian Chemical Society) Publications       ACS Chemical 

In [416]:
# Create column that is a count of journal articles with that journal name for each publisher
df_Article_title_count_by_journal_publisher = df.groupby(
    ["Journal title", "Publisher"])['Article title'].count()

## Answer: 1. Five most common journals and total articles:

In [417]:
# Five most common journals by article number
df_Article_title_count_by_journal_publisher.groupby('Journal title').sum().nlargest(5)

Journal title
PLOS One                                           209
Journal of Biological Chemistry                     48
Nucleic Acids Research                              29
Proceedings of the National Academy of Sciences     19
Human Molecular Genetics                            18
Name: Article title, dtype: int64

### Mean, median, and standard deviation of the open-access cost per article for each journal

In [418]:
# Remove rows that have text in 'Cost' column
df[df['Cost'].map(lambda x: str(x)!='[abcdefghijklmnopqrstuvwxyz]')].head()

,Publisher_id,Publisher,Journal title,Article title,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,ﾣ0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,ﾣ2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",ﾣ642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,ﾣ669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,ﾣ685.88


In [419]:
# Remove rows with implausible 'Cost' = 999999.00
df[df['Cost'].map(lambda x: str(x)!='ﾣ999999.00')].head()

,Publisher_id,Publisher,Journal title,Article title,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,ﾣ0.00
1,PMC3679557,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,ﾣ2381.04
2,23043264 PMC3506128,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",ﾣ642.56
3,23438330 PMC3646402,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,ﾣ669.64
4,23438216 PMC3601604,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,ﾣ685.88


In [424]:
# Remove ﾣ symbol from all Cost entries by extracting the digits from the entry
# Regex \more than one digit; then a literal .; then one digit; next digit
df['Cost'] = df['Cost'].str.extract('(\d+\.\d\d)').astype(float)

In [425]:
# Is it working? Yes
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(df.groupby(['Journal title']).agg({'Cost': 'sum'}))

                                                          Cost
Journal title                                                 
ACS Chemical Biology                                   7090.93
ACS Chemical Neuroscience                              1186.80
ACS NANO                                                642.89
ACS Nano                                                693.39
ACTA F                                                  754.90
AGE                                                    2002.00
AIDS                                                   6177.92
AIDS Behav                                             1834.77
AIDS Care                                              4378.34
AIDS Journal                                           2015.72
AIDS Research and Therapy                              1240.00
AIDS UK                                                3673.84
ASN Neuro                                              1453.42
Academy of Nutrition and Dietetics                     

## Answer 2: Mean cost of articles

In [426]:
df['Cost'].mean()

24061.2280771039

## Answer 2: Median cost of articles

In [428]:
df['Cost'].median()

1883.71

## Answer 2: STD of article cost

In [430]:
df['Cost'].std()

146861.62698444453

In [439]:
# Brute combination of Publishers and Journals below:

In [145]:
# Check to make sure all journal titles combined that can be combined.  All Plos One variations combined.  check!
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(df.groupby(['Journal title', 'Publisher']).agg({'Journal title': 'count'}))

                                                                                                       Journal title
Journal title                                      Publisher                                                        
ACS Chemical Biology                               ACS (Amercian Chemical Society) Publications                    2
                                                   American Chemical Society                                       3
ACS Chemical Neuroscience                          American Chemical Society                                       1
ACS NANO                                           AMERICAN CHEMICAL SOCIETY                                       1
ACS Nano                                           American Chemical Society Publications                          1
ACTA F                                             International Union of Crystallography                          1
AGE                                                Springer-Verl

In [146]:
#Just to clean things up abit.
# Determine unique publishers
df['Journal title'].unique()

array(['Psychological Medicine', 'Biomacromolecules', 'J Med Chem',
       'J Org Chem', 'Journal of Medicinal Chemistry',
       'Journal of Proteome Research', 'Mol Pharm',
       'ACS Chemical Biology',
       'Journal of Chemical Information and Modeling', 'Biochemistry',
       'Gastroenterology', 'Journal of Biological Chemistry',
       'Journal of Immunology', 'ACS Chemical Neuroscience', 'ACS NANO',
       'American Chemical Society', 'Analytical Chemistry',
       'Bioconjugate Chemistry', 'Journal of Medicinal Chemistry ',
       'Journal of the American Chemical Society', 'ACS Nano', 'CHEST',
       'Journal of Neurophysiology', 'Journal of Physiology',
       'The Journal of Neurophysiology', 'American Journal of Psychiatry',
       'Americal Journal of Psychiatry', 'Behavioral Neuroscience',
       'Emotion', 'Health Psychology', 'Journal of Abnormal Psychology',
       'Journal of Consulting and Clinical Psychology',
       'Journal of Experimental Psychology:  Animal Be

In [147]:
# Get value counts to isolate where to focus data cleaning - just pick top X number to focus on cleaning since everything below is irrelvant
df['Publisher'].value_counts()

Elsevier                                                   377
Public Library of Science                                  263
Wiley                                                      133
Springer                                                    81
Oxford University Press                                     75
Wiley-Blackwell                                             56
OUP                                                         55
ASBMB                                                       45
Nature Publishing Group                                     44
BioMed Central                                              40
BMC                                                         25
BMJ                                                         23
Royal Society                                               22
Nature                                                      22
Frontiers                                                   22
Cambridge University Press                             

## Eliminate white spaces

In [148]:
# Strip all white-spaces out of publisher to eliminate this source of incorrect unique name across publishers
df['Publisher'] = df['Publisher'].str.strip()
df['Journal title'] = df['Journal title'].str.strip()

## Identify/handle missing values

In [149]:
# Return all rows with missing values
df[df.isnull().any(axis=1)]

,Publisher_id,Publisher,Journal title,Article title,Cost
0,NaN,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,ﾣ0.00
21,NaN,American Chemical Society,ACS Chemical Biology,Discovery of ?2 Adrenergic Receptor Ligands Us...,ﾣ947.07
43,NaN,American Psychiatric Association,American Journal of Psychiatry,Methamphetamine-induced disruption of frontost...,ﾣ2351.73
93,NaN,American Society for Microbiology,Journal of Virology,The human adenovirus type 5 L4 promoter is act...,ﾣ1312.59
100,NaN,American Society for Nutrition,American Society for Nutrition,The association between breastfeeding and HIV ...,ﾣ45.94
101,NaN,American Society of Haematology,Blood,Disease-associated missense mutations in the E...,ﾣ1260.26
102,NaN,American Society of Haematology,Blood,SAP gene transfer restores cellular and humora...,ﾣ1260.26
104,NaN,American Society of Hematology,Blood,Super-resolution imaging of remodeled synaptic...,ﾣ1230.40
114,NaN,American Speech-Language-Hearing Association,Speech Language and Hearing Research,Developmental Trajectories of Verbal and Nonve...,ﾣ1969.02
126,NaN,ASBMB,Journal of Biological Chemistry,Matrilin-1 A-domains: structural and functiona...,ﾣ1260.90


In [150]:
# Fill NaN with specified value
NaN_fill = {'Publisher_id': '0000', 'Journal title': 'Missing_title'}
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df.fillna(value=NaN_fill))


                                           Publisher_id  \
0                                                  0000   
1                                            PMC3679557   
2                                 23043264  PMC3506128    
3                                   23438330 PMC3646402   
4                                  23438216 PMC3601604    
5                                            PMC3579457   
6                                            PMC3709265   
7                                  23057412 PMC3495574    
8                                     PMCID: PMC3780468   
9                                     PMCID: PMC3621575   
10                                    PMCID: PMC3739413   
11                                    PMCID: PMC3530961   
12                                    PMCID: PMC3624797   
13                                           PMC3413243   
14                                           PMC3694353   
15                                           PMC3572711 

### Combine all publishers to be able to compare journals and eliminate unique differences in journal title due to spelling and not due to actually being a unique journal title

In [151]:
# Find all publishers that contain ACS in string and change to "American Chemical Society"
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('ACS'), 'Publisher'] = 'American Chemical Society'

# Combine American Chemical Society: make it case sensitive
df.loc[df['Publisher'].str.contains('AMERICAN CHEMICAL SOCIETY', case=False), 'Publisher'] = 'American Chemical Society'

In [152]:
#It worked
df.groupby(['Publisher', 'Journal title', 'Publisher_id']).agg({'Journal title': 'count'})

Journal title
Publisher                          Journal title                                      Publisher_id                                   
AGA Institute                      Gastroenterology                                   PMC3572711                                    1
AMBSB                              Journal of Biological Chemistry                    22610094                                      1
ASBMB                              BIOLOGICAL CHEMISTRY                               PMCID: 3581401                                1
                                   Biologicial Chemistry                              PMC3591638                                    1
                                   J Biol Chem.                                       PMC3265891                                    1
                                                                                      PMC3308871                                    1
                                                                                      PMC3438964                                    1
                                   J Biol Chemistry                                   PMC3597819                                    1
                                   JOURNAL OF BIOLOGICAL CHEMISTRY                    PMC3567677                                    1
                                                                                      PMCID: 3682544                                1
                                   Journal of Biological Chemistry                     PMC3516716                                   1
                                                                                       PMCID: PMC3750171                            1
                                                                                      23129765 PMC3531760                           1
                                                                                      2339840 PMC3591641                            1
                                                                                      23569211 PMC3668706                           1
                                                                                      2797226                                       1
                                                                                      2898433                                       1
                                                                                      2952269                                       1
                                                                                      2975197                                       1
                                                                                      3121340                                       1
                                                                                      3137072                                       1
                                                                                      3234717                                       1
                                                                                      3397837                                       1
                                                                                      3431625                                       1
                                                                                      3696646                                       1
                                                                                      : PMC3711325                                  1
                                                                                      PMC2757965                                    1
                                                                                      PMC3173209                                    1
                                                                                      PMC3442528    

In [153]:
# Convert mis-spelling AMBSB to "ASBMB"
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('AMBSB'), 'Publisher'] = 'ASBMB'
df.loc[df['Publisher'].str.contains('ASBMC'), 'Publisher'] = 'ASBMB'

# Combine all ASBMB
df.loc[df['Publisher'].str.contains('ASBMB'), 'Publisher'] = 'ASBMB'

In [154]:
# Combine all Wiley
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Wiley', case=False), 'Publisher'] = 'Wiley'
df.loc[df['Publisher'].str.contains('Wliey-Blackwell'), 'Publisher'] = 'Wiley'


In [155]:
# Combine all Wolters
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Wolters'), 'Publisher'] = 'Wolters'

In [156]:
# Combine all American Psychiatric Publishing
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('American Psychiatric'), 'Publisher'] = 'American Psychiatric Publishing'

In [157]:
# Combine all Biochemistry and Molecular Biology
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Biochemistry and Molecular'), 'Publisher'] = 'American Society for Biochemistry and Molecular Biology'

In [158]:
# Combine all American Society of Hematology
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('American Society of Haematology'), 'Publisher'] = 'American Society of Hematology'
df.loc[df['Publisher'].str.contains('American Society of Hamatology'), 'Publisher'] = 'American Society of Hematology'

In [159]:
# Combine all BMJ
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('BMJ'), 'Publisher'] = 'British Medical Journal'

In [160]:
# Correct spelling per internet verification to Bentham Science Publishers
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Benthan Science Publishers'), 'Publisher'] = 'Bentham Science Publishers'

In [161]:
# Combine all biomed
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Biomed', case=False), 'Publisher'] = 'BioMed Central'
df.loc[df['Publisher'].str.contains('BMC', case=False), 'Publisher'] = 'BioMed Central'

In [162]:
# Combine all RSC
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('RSC', case=False), 'Publisher'] = 'RSC'

In [163]:
# Combine all ASM
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('ASM', case=False), 'Publisher'] = 'American Society for Microbiology'

In [164]:
# Combine all BioScientifica
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('BioScientifica', case=False), 'Publisher'] = 'BioScientifica'

In [165]:
# Combine all Springer
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Springer', case=False), 'Publisher'] = 'Springer'

In [166]:
# Combine all Sage
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Sage', case=False), 'Publisher'] = 'Sage'

In [167]:
# Combine all Biophysical Society
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Byophysical Society', case=False), 'Publisher'] = 'Biophysical Society'

In [168]:
# Combine all Royal Society for Chemistry
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Royal Society of Chemistry', case=False), 'Publisher'] = 'Royal Society for Chemistry'

In [169]:
# Combine all Taylor and Francis
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Taylor', case=False), 'Publisher'] = 'Taylor & Francis'
df.loc[df['Publisher'].str.contains('T&F', case=False), 'Publisher'] = 'Taylor & Francis'

In [170]:
# Combine all American Physiological Society
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('American Physiological Society', case=False), 'Publisher'] = 'American Physiological Society'

In [171]:
# Combine all The Endrocrine Society and correct spelling
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Endocrine Soc', case=False), 'Publisher'] = 'The Endocrine Society'
df.loc[df['Publisher'].str.contains('The Endrocrine', case=False), 'Publisher'] = 'The Endocrine Society'

In [172]:
# Combine all The Company of Biologists
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('The Company of Biologists', case=False), 'Publisher'] = 'The Company of Biologists'
df.loc[df['Publisher'].str.contains('The Company of Biolgists', case=False), 'Publisher'] = 'The Company of Biologists'

In [173]:
# Combine all The Royal Society
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('The Royal Society', case=False), 'Publisher'] = 'Royal Society'

In [174]:
# Combine all Society for Neuroscience
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Society for Neuroscience', case=False), 'Publisher'] = 'Society for Neuroscience'
df.loc[df['Publisher'].str.contains('Society of Neuroscience', case=False), 'Publisher'] = 'Society for Neuroscience'
df.loc[df['Publisher'].str.contains('SOCIETY OF NEURO SCIENCES', case=False), 'Publisher'] = 'Society for Neuroscience'


In [175]:
# Combine all Society for Leukocyte Biology per internet search is "for"
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Society of Leukocyte Biology', case=False), 'Publisher'] = 'Society for Leukocyte Biology'


In [176]:
# Combine all Society for Genermal Microbiology
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Genermal Microbiology', case=False), 'Publisher'] = 'Society for General Microbiology'
df.loc[df['Publisher'].str.contains('Society of General Microbiology', case=False), 'Publisher'] = 'Society for General Microbiology'


In [177]:
# Combine all CADMUS JOURNAL SERVICE
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('CADMUS', case=False), 'Publisher'] = 'Cadmus Journal Service'
df.loc[df['Publisher'].str.contains('CAMdUS', case=False), 'Publisher'] = 'Cadmus Journal Service'


In [178]:
# Combine all Cambridge
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Cambridge', case=False), 'Publisher'] = 'Cambridge University Press'


In [179]:
# Combine all Cold Harbor
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Cold Spring', case=False), 'Publisher'] = 'Cold Spring Harbor Press'


In [180]:
# Combine all Cold Harbor
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Company of Biologist', case=False), 'Publisher'] = 'Company of Biologists'

In [181]:
# Combine all Dartmouth Journal Services
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Dartmouth', case=False), 'Publisher'] = 'Dartmouth Journal Services'
df.loc[df['Publisher'].str.contains('Darmouth', case=False), 'Publisher'] = 'Dartmouth Journal Services'

In [182]:
# Combine all Elsevier
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Elsevier', case=False), 'Publisher'] = 'Elsevier'
df.loc[df['Publisher'].str.contains('Elseveier', case=False), 'Publisher'] = 'Elsevier'

In [183]:
# Combine all FASEB
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Experimental Biology', case=False), 'Publisher'] = 'FASEB'

In [184]:
# Combine all Frontiers
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Frontiers', case=False), 'Publisher'] = 'Frontiers'

In [185]:
# Combine all Hindawi
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Hindawi', case=False), 'Publisher'] = 'Hindawi'

In [186]:
# Combine all Future Medicine
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Future Medicine', case=False), 'Publisher'] = 'Future Medicine'

In [187]:
# Combine all Impact Journals
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Impact Journals', case=False), 'Publisher'] = 'Impact Journals'

In [188]:
# Combine all Informa Healthcare
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Informa Healthcare', case=False), 'Publisher'] = 'Informa Healthcare'

In [189]:
# Combine all International Union of Crystallography
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Union Against', case=False), 'Publisher'] = 'International Union Against Tuberculosis'

In [190]:
# Combine all International Union Against Tuberculosis
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('International Union of Crystallography ', case=False), 'Publisher'] = 'International Union of Crystallography'

In [191]:
# Combine all Karger
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Karger', case=False), 'Publisher'] = 'Karger'

In [192]:
# Combine all Jove
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Jove', case=False), 'Publisher'] = 'Jove'

In [193]:
# Combine all MIT
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('MIT', case=False), 'Publisher'] = 'MIT Press'

In [194]:
# Combine all Landes
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Landes', case=False), 'Publisher'] = 'Landes Press'

In [195]:
# Combine all National Academy of Sciences USA
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('National Academy of Sciences', case=False), 'Publisher'] = 'National Academy of Sciences USA'

In [196]:
# Combine all Mary Ann Liebert
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Mary Ann Liebert', case=False), 'Publisher'] = 'Mary Ann Liebert'

In [197]:
# Combine all Nature Publishing
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Nature', case=False), 'Publisher'] = 'Nature Publishing'

In [198]:
# Combine all OXFORD UNIVERSITY PRESS 
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Oup', case=False), 'Publisher'] = 'Oxford University Press'
df.loc[df['Publisher'].str.contains('OXFORD', case=False), 'Publisher'] = 'Oxford University Press'

In [199]:
# Combine all PLOS public library of science Public Library of Science
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('PLOS', case=False), 'Publisher'] = 'Public Library of Science'
df.loc[df['Publisher'].str.contains('Public Library of Science', case=False), 'Publisher'] = 'Public Library of Science'

In [200]:
# Combine all PNAS public library of science
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('PNAS', case=False), 'Publisher'] = 'PNAS'

In [201]:
# Combine all MacMillan
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('MacMillan', case=False), 'Publisher'] = 'MacMillan'

In [202]:
# Combine all Portland Press
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Portland Press', case=False), 'Publisher'] = 'Portland Press'

In [203]:
# Combine all PubMed
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('PubMed', case=False), 'Publisher'] = 'PubMed'

In [204]:
# Combine all Sheridan
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('Sheridan', case=False), 'Publisher'] = 'Sheridan Press'

In [205]:
# Combine all Royal College of Psychiatrists
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('The Royal College of Psychiatrists', case=False), 'Publisher'] = 'Royal College of Psychiatrists'

In [206]:
# Combine all American Society for Microbiology
# (case=False to make case-insensitive)
df.loc[df['Publisher'].str.contains('American Society of Microbiology', case=False), 'Publisher'] = 'American Society for Microbiology'

In [207]:
# Re-check to make sure all journal titles combined that can be combined.
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(df.groupby(['Publisher', 'Journal title']).agg({'Publisher': 'count'}))

                                                                                                       Publisher
Publisher                                          Journal title                                                
AGA Institute                                      Gastroenterology                                            1
ASBMB                                              BIOLOGICAL CHEMISTRY                                        1
                                                   Biologicial Chemistry                                       1
                                                   J Biol Chem.                                                3
                                                   J Biol Chemistry                                            1
                                                   JOURNAL OF BIOLOGICAL CHEMISTRY                             2
                                                   Jnl Biological Chemistry                     

In [208]:
## Combine Journals under publishers

In [209]:
# Combine ASBMB biological chemistry
df.loc[df['Journal title'].str.contains('biol', case=False, na=False), 'Journal title'] = 'Journal of Biological Chemistry'

In [210]:
# Combine ACS NANO
df.loc[df['Journal title'].str.contains('ACS NANO', case=False, na=False), 'Journal title'] = 'ACS Nano'

In [211]:
# Combine J Med Chem
df.loc[df['Journal title'].str.contains('American Chemical Society', case=False, na=False), 'Journal title'] = 'Journal of Medicinal Chemistry'



In [212]:
# Combine J Med Chem
df.loc[df['Journal title'].str.contains('J Med Chem', case=False, na=False), 'Journal title'] = 'Journal of the American Chemical Society'


In [213]:
# Combine Biochem
df.loc[df['Journal title'].str.contains('Journal of Biological Chemistry', case=False, na=False), 'Journal title'] = 'Biochemistry'


In [214]:
# Combine Psychiatry
df.loc[df['Journal title'].str.contains('Americal Journal of Psychiatry', case=False, na=False), 'Journal title'] = 'American Journal of Psychiatry'


In [215]:
# Combine Psychiatry
df.loc[df['Journal title'].str.contains('Journal of Virol', case=False, na=False), 'Journal title'] = 'Journal of Virology'


In [216]:
# Combine Hematology
df.loc[df['Journal title'].str.contains('Blood', case=False, na=False), 'Journal title'] = 'Blood'


In [217]:
# Combine BMC Genomics
df.loc[df['Journal title'].str.contains('BMC Genomics', case=False, na=False), 'Journal title'] = 'BMC Genomics'
df.loc[df['Journal title'].str.contains('BMs Genomics', case=False, na=False), 'Journal title'] = 'BMC Genomics'


In [218]:
# Combine BMC Genomics
df.loc[df['Journal title'].str.contains('BMC Infectious diseases', case=False, na=False), 'Journal title'] = 'BMC Infectious Diseases'


In [219]:
# Combine BMC International Journal of Behavioural Nutrition
df.loc[df['Journal title'].str.contains('International Journal of Behavioural Nutrition', case=False, na=False), 'Journal title'] = 'International Journal of Behavioral Nutrition and Physical Activity'


In [220]:
# Combine Parasite Vectors
df.loc[df['Journal title'].str.contains('Parasit Vectors.', case=False, na=False), 'Journal title'] = 'Parasite Vectors'
df.loc[df['Journal title'].str.contains('Parasites and Vectors', case=False, na=False), 'Journal title'] = 'Parasite Vectors'

In [221]:
# Combine Parasite Vectors
df.loc[df['Journal title'].str.contains('RESPIRATORY RESEARCH', case=False, na=False), 'Journal title'] = 'Respiratory Research'

In [222]:
# Combine Hematology
df.loc[df['Journal title'].str.contains('Journal of molecular Endocrinology', case=False, na=False), 'Journal title'] = 'Journal of Molecular Endocrinology'


In [223]:
# Combine Human Resources Health
df.loc[df['Journal title'].str.contains('Hum Resour Health', case=False, na=False), 'Journal title'] = 'Human Resources for Health'

In [224]:
# Combine BMJ Open
df.loc[df['Journal title'].str.contains('BMJ open', case=False, na=False), 'Journal title'] = 'BMJ Open'


In [225]:
# Combine BMJ
df.loc[df['Journal title'].str.contains('British Medical Journal', case=False, na=False), 'Journal title'] = 'BMJ'


In [226]:
# Combine BMJ British Journal of Opthalmology
df.loc[df['Journal title'].str.contains('British Journal of Ophthalmology', case=False, na=False), 'Journal title'] = 'British Journal of Opthalmology'


In [227]:
# Combine BMJ
df.loc[df['Journal title'].str.contains('Journal of Epidemiology and Community Health', case=False, na=False), 'Journal title'] = 'Journal of Epidemiology & Community Health'


In [228]:
# Combine BMJ
df.loc[df['Journal title'].str.contains('Sex Transm Infect.', case=False, na=False), 'Journal title'] = 'Sexually Transmitted Infections'


In [229]:
# Combine BMJ thorax
df.loc[df['Journal title'].str.contains('THORAX', case=False, na=False), 'Journal title'] = 'Thorax'


In [230]:
# Combine British Journal for the History of Science 
df.loc[df['Journal title'].str.contains('British Journal of the History of Science', case=False, na=False), 'Journal title'] = 'British Journal for the History of Science'


In [231]:
# Combine Psychological Medicine
df.loc[df['Journal title'].str.contains('Pyschological Medicine', case=False, na=False), 'Journal title'] = 'Psychological Medicine'


In [232]:
# Combine Development
df.loc[df['Journal title'].str.contains('Developmental Cell', case=False, na=False), 'Journal title'] = 'Development'


In [233]:
# Combine Development
df.loc[df['Journal title'].str.contains('J Cell Sci.', case=False, na=False), 'Journal title'] = 'Journal of Cell Science'


In [234]:
# Re-check to make sure all publishers combined that can be combined.
with pd.option_context('display.max_rows', None, 'display.max_columns', 3):
    print(df.groupby(['Publisher', 'Journal title']).agg({'Publisher': 'count'}))

                                                                                                       Publisher
Publisher                                          Journal title                                                
AGA Institute                                      Gastroenterology                                            1
ASBMB                                              Biochemistry                                               48
                                                   MCP (Molecular & Cellular Proteomics)                       1
                                                   Molecular and Cellular Proteomics                           1
American Association of Immunologists              Journal of Immunology                                       2
American Chemical Society                          ACS Chemical Neuroscience                                   1
                                                   ACS Nano                                     

In [235]:
df['Publisher'].value_counts()

Elsevier                                                   399
Public Library of Science                                  292
Wiley                                                      266
Oxford University Press                                    164
Springer                                                    95
BioMed Central                                              93
Nature Publishing                                           77
British Medical Journal                                     58
ASBMB                                                       50
American Chemical Society                                   34
Frontiers                                                   31
Royal Society                                               29
National Academy of Sciences USA                            27
Company of Biologists                                       26
American Society for Microbiology                           25
Cambridge University Press                             

In [433]:
df['Journal title'].value_counts()

PLOS One                                                         209
Journal of Biological Chemistry                                   48
Nucleic Acids Research                                            29
Proceedings of the National Academy of Sciences                   19
PLoS Neglected Tropical Diseases                                  18
Human Molecular Genetics                                          18
Nature Communications                                             17
PLoS Pathogens                                                    15
PLoS Genetics                                                     15
Neuroimage                                                        15
BMC Public Health                                                 14
NeuroImage                                                        14
Brain                                                             14
Movement Disorders                                                13
Biochemical Journal               

Regex:
    https://regexr.com/
